# Obtenção dos dados

#### Separamos esse trecho do código para deixar mais organizado, além de não precisar fazer a extração dos dados diretamente do MYSQL em cada execução do código principal.

Escolhemos trabalhar com a base de dados criada para a A1, que contém dados nutricionais de lanches de três redes de fast food distintas: Mc Donalds, Burguer King e Starbucks. O primeiro passo, então, foi carregar os dados que estavam no MySQL para o Python utilizando as bibliotecas Pandas e Pymysql.

In [1]:
import pandas as pd
import pymysql
import getpass


Criação da conexão com o banco local:

In [2]:
p = getpass.getpass()
conexao = pymysql.connect(
    host='localhost',
    user='root',
    port=3306,
    passwd=p,
    db="fast_food")

········


Queremos fazer uma visualização que permita comparar os valores nutricionais dos lanches (através do tamanho das arestas), para isso importamos todos os lanches, sua tabela nutricional, a rede a que pertence e sua categoria. Esses últimos serão úteis para filtrar os dados.

In [3]:
query = """
SELECT p.name, n.*, c.name as "categoria", p.chain_id_chain as "rede"
FROM nutricional_units n
INNER JOIN product p ON n.id_nutricional_units = p.nutricional_units_id_nutricional_units
INNER JOIN category c ON p.category_id_category = c.id_category;
"""

df = pd.read_sql(query, con=conexao, index_col = "name")

# Tratamento dos dados

#### Para que a visualização do grafo ficasse da maneira que planejamos, precisamos tratar os dados obtidos.

Queremos que o tamanho (grossura) da aresta represente a quantidade presente do nutriente no lanche conectado. Como o range dentre os diferentes nutrientes é muito diferente, precisamos normalizar os dados para melhor visualização:

In [4]:
col=[x for x in df.columns[0:len(df.columns)-2]] #pega os nomes das colunas de nutrientes (-2 exclui "categoria" e "rede")
nutrientes = df[col].copy() # criar uma cópia, somente com as colunas dos nutrientes

nutri_norm = (nutrientes - nutrientes.mean()) / nutrientes.std() #normaliza os nutrientes
nutri_min = abs(nutri_norm.min())

nutri_norm = nutri_norm + nutri_min + 1 #deixar todos os valores > 0

for nutriente in col:
    df[nutriente] = nutri_norm[nutriente] # aplica a normalização feita no df original

Iremos representar as redes pelas suas cores características, para facilitar o processo, trocamos cada nome no dataframe pela cor que queremos aplicar:

In [5]:
#atribuir cor de acordo com as redes
cores = {1:"#0055AA",2:"#CF1C24",3:"#24704D"}  

df.replace({'rede': cores},inplace=True)

Os NA podem ser trocados por 0 no nosso problema, pois de qualquer modo não serão representados pelos nodes. Caso fossem ignorados todos os dados que possuem NA, perderíamos muitas informações, pois muitos lanches seriam excluidos da base.

In [6]:
df.fillna(0,inplace=True)

Por fim, simplismente renomeamos as colunas dos nutrientes, pois serão os nomes utilizados nos nodes e no banco estão em inglês, além de alguns contem "_", o que não seria agradável na visualização do grafo.

In [7]:
# 'col' foi definido acima e seleciona somente as colunas dos nutrientes
col_names = ['calorias','gorduras totais','gorduras saturadas','gorduras trans','colesterol','sódio','carboidratos','fibras','açúcares','proteínas','vitamina A','vitamina C','cálcio','ferro']

df = df.rename(columns=dict(zip(col,col_names))) #renomeia as colunas dos nutrientes com os novos nomes (na lista col_names)

# Resultado

#### Após esses passos iniciais obtemos o seguinte data frame:

In [8]:
df

,calorias,gorduras totais,gorduras saturadas,gorduras trans,colesterol,sódio,carboidratos,fibras,açúcares,proteínas,vitamina A,vitamina C,cálcio,ferro,iron,salt,caffeine,categoria,rede
name,,,,,,,,,,,,,,,,,,,
WHOPPER,1.000000,3.505575,2.984077,2.897831,1.0,0.000000,1.002474,1.214283,1.371581,0.000000,3.481996,0.000000,0.0,0.000000,0.0,0.0,0.0,WHOPPER,#0055AA
WHOPPER DUPLO,1.004561,4.612173,4.287898,3.696918,1.0,0.000000,1.002474,1.198410,1.557372,0.000000,5.226101,0.000000,0.0,0.000000,0.0,0.0,0.0,WHOPPER,#0055AA
WHOPPER FURIOSO,1.009122,4.101435,3.380892,2.797945,1.0,0.000000,1.004948,1.234124,1.371581,0.000000,3.616158,0.000000,0.0,0.000000,0.0,0.0,0.0,WHOPPER,#0055AA
WHOPPER JR.,1.013683,2.365668,2.020382,1.099886,1.0,0.000000,2.561174,1.126982,1.371581,0.000000,2.408700,0.000000,0.0,0.000000,0.0,0.0,0.0,WHOPPER,#0055AA
WHOPPER JR. BARBECUE BACON,1.018244,2.654346,2.247134,1.998859,1.0,0.000000,3.145068,1.146823,1.371581,0.000000,2.811186,0.000000,0.0,0.000000,0.0,0.0,0.0,WHOPPER,#0055AA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SWEETENED WHIPPED CREAM SHORT - HOT,4.434456,1.185050,1.283440,1.299658,1.0,1.321372,1.012371,1.003968,1.000000,1.041546,1.000000,1.345003,1.0,1.000000,1.0,0.0,1.0,OTHERS BEVERAGE,#24704D
SWEETENED WHIPPED CREAM TALL - HOT,4.439017,1.222060,1.340127,1.399543,1.0,1.321372,1.012371,1.007936,1.000000,1.041546,1.000000,1.345003,1.0,1.000000,1.0,0.0,1.0,OTHERS BEVERAGE,#24704D
SWEETENED WHIPPED CREAM GRANDE/VENTI® - HOT,4.443578,1.259070,1.396815,1.499429,1.0,1.401715,1.012371,1.007936,1.000000,1.083092,1.000000,1.460004,1.0,1.119992,1.0,0.0,1.0,OTHERS BEVERAGE,#24704D


# Exportação do data frame

In [9]:
#df.to_csv("fastfood.csv")